In [ ]:
#http://alexminnaar.com/2019/08/22/ner-rnns-tensorflow.html
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import pandas as pd
import numpy as np
import os
import time
from sklearn.model_selection import train_test_split

print(tf.__version__)

2.5.0


In [ ]:
df = pd.read_csv("entities_final.csv")

In [ ]:
df.head()

,question,entities
0,who's the writer of Experiment Perilous,"['0', '0', '0', '0', 'B-movie', 'I-movie']"
1,who's the writer of Games,"['0', '0', '0', '0', 'B-movie']"
2,which person wrote The Wolf Man,"['0', '0', '0', 'B-movie', 'I-movie', 'I-movie']"
3,who is the writer of the film Sweet Charity,"['0', '0', '0', '0', '0', '0', '0', 'B-movie',..."
4,who was the writer of Extract,"['0', '0', '0', '0', '0', 'B-movie']"


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
         df["question"], df["entities"], test_size=0.5, random_state=2021)

X_val, X_test, y_val, y_test = train_test_split(
         X_test, y_test, test_size=0.5, random_state=2021)

In [ ]:
labels = set([label for line in df["entities"] for label in eval(line)])

In [ ]:
all_text = " ".join([" ".join(question) for question in df["question"]])
vocab = sorted(set(all_text))

In [ ]:
# create character/id and label/id mapping
char2idx = {u:i+1 for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
label2idx = {u:i+1 for i, u in enumerate(labels)}
idx2label = np.array(labels)

print(idx2label)
print(char2idx)

{'I-actor', 'I-director', 'I-movie', 'I-writer', 'I-tag', 'B-writer', 'B-actor', 'B-tag', 'B-director', 'B-movie', '0'}
{' ': 1, '!': 2, "'": 3, '(': 4, ')': 5, '+': 6, ',': 7, '-': 8, '.': 9, '/': 10, '0': 11, '1': 12, '2': 13, '3': 14, '4': 15, '5': 16, '6': 17, '7': 18, '8': 19, '9': 20, 'A': 21, 'B': 22, 'C': 23, 'D': 24, 'E': 25, 'F': 26, 'G': 27, 'H': 28, 'I': 29, 'J': 30, 'K': 31, 'L': 32, 'M': 33, 'N': 34, 'O': 35, 'P': 36, 'Q': 37, 'R': 38, 'S': 39, 'T': 40, 'U': 41, 'V': 42, 'W': 43, 'X': 44, 'Y': 45, 'Z': 46, 'a': 47, 'b': 48, 'c': 49, 'd': 50, 'e': 51, 'f': 52, 'g': 53, 'h': 54, 'i': 55, 'j': 56, 'k': 57, 'l': 58, 'm': 59, 'n': 60, 'o': 61, 'p': 62, 'q': 63, 'r': 64, 's': 65, 't': 66, 'u': 67, 'v': 68, 'w': 69, 'x': 70, 'y': 71, 'z': 72, '¡': 73, '½': 74, 'À': 75, 'É': 76, 'Õ': 77, 'à': 78, 'á': 79, 'â': 80, 'ã': 81, 'ä': 82, 'å': 83, 'æ': 84, 'ç': 85, 'è': 86, 'é': 87, 'ê': 88, 'ì': 89, 'í': 90, 'î': 91, 'ï': 92, 'ñ': 93, 'ò': 94, 'ó': 95, 'ô': 96, 'ö': 97, 'ø': 98, 'û': 9

In [ ]:
def split_char_labels(X, y):
    '''
    For a given input/output example, break tokens into characters while keeping 
    the same label.
    '''

    tokens = X
    labels = eval(y)

    input_chars = []
    output_char_labels = []

    for token,label in zip(tokens,labels):

        input_chars.extend([char for char in token])
        input_chars.extend(' ')
        output_char_labels.extend([label]*len(token))
        output_char_labels.extend('0')

    return [[char2idx[x] for x in input_chars[:-1]],np.array([label2idx[x] for x in output_char_labels[:-1]])]

train_formatted = list(map(split_char_labels, X_train, y_train))
test_formatted = list(map(split_char_labels, X_test, y_test))
valid_formatted = list(map(split_char_labels, X_val, y_val))

print(len(train_formatted))
print(len(test_formatted))
print(len(valid_formatted))

4999
2500
2500


In [ ]:
len(train_formatted[1][1])

7

In [ ]:
# training generator
def gen_train_series():
    for eg in train_formatted:
        yield eg[0],eg[1]
    
    # validation generator
def gen_valid_series():

    for eg in valid_formatted:
        yield eg[0],eg[1]
    
# test generator
def gen_test_series():

    for eg in test_formatted:
        yield eg[0],eg[1]
      
# create Dataset objects for train, test and validation sets  
series = tf.data.Dataset.from_generator(gen_train_series,output_types=(tf.int32, tf.int32),output_shapes = ((None, None)))
series_valid = tf.data.Dataset.from_generator(gen_valid_series,output_types=(tf.int32, tf.int32),output_shapes = ((None, None)))
series_test = tf.data.Dataset.from_generator(gen_test_series,output_types=(tf.int32, tf.int32),output_shapes = ((None, None)))

BATCH_SIZE = 128
BUFFER_SIZE=1000
    
# create padded batch series objects for train, test and validation sets
ds_series_batch = series.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, padded_shapes=([None], [None]), drop_remainder=True)
ds_series_batch_valid = series_valid.padded_batch(BATCH_SIZE, padded_shapes=([None], [None]), drop_remainder=True)
ds_series_batch_test = series_test.padded_batch(BATCH_SIZE, padded_shapes=([None], [None]), drop_remainder=True)

# print example batches
for input_example_batch, target_example_batch in ds_series_batch_valid.take(1):
    print(input_example_batch)
    print(target_example_batch)


tf.Tensor(
[[69  1 54 ...  0  0  0]
 [69  1 54 ...  0  0  0]
 [69  1 54 ...  0  0  0]
 ...
 [69  1 54 ...  0  0  0]
 [69  1 54 ...  0  0  0]
 [69  1 54 ...  0  0  0]], shape=(128, 27), dtype=int32)
tf.Tensor(
[[11 11 11 ...  0  0  0]
 [11 11 11 ...  0  0  0]
 [11 11 11 ...  0  0  0]
 ...
 [11 11 11 ...  0  0  0]
 [11 11 11 ...  0  0  0]
 [11 11 11 ...  0  0  0]], shape=(128, 27), dtype=int32)


In [ ]:
  vocab_size = len(vocab)+1

  # The embedding dimension
  embedding_dim = 256

  # Number of RNN units
  rnn_units = 1024

  label_size = len(labels)  
  
  # build LSTM model
  def build_model(vocab_size,label_size, embedding_dim, rnn_units, batch_size):
        model = tf.keras.Sequential([
            tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None],mask_zero=True),
            tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True, # a saida da lstm vai ser um conj de vetores
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
            tf.keras.layers.Dense(label_size)
            ])
        return model

  model = build_model(
        vocab_size = len(vocab)+1,
        label_size=len(labels)+1,
        embedding_dim=embedding_dim,
        rnn_units=rnn_units,
        batch_size=BATCH_SIZE)

  model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 256)          25856     
_________________________________________________________________
lstm (LSTM)                  (128, None, 1024)         5246976   
_________________________________________________________________
dense (Dense)                (128, None, 12)           12300     
Total params: 5,285,132
Trainable params: 5,285,132
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import os

# define loss function
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss, metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
EPOCHS=20
history = model.fit(ds_series_batch, epochs=EPOCHS, validation_data=ds_series_batch_valid,callbacks=[checkpoint_callback])

Epoch 1/20
39/39 [==============================] - 27s 616ms/step - loss: 0.4983 - sparse_categorical_accuracy: 0.7887 - val_loss: 0.3418 - val_sparse_categorical_accuracy: 0.8234
Epoch 2/20
39/39 [==============================] - 23s 576ms/step - loss: 0.3281 - sparse_categorical_accuracy: 0.8228 - val_loss: 0.2751 - val_sparse_categorical_accuracy: 0.8367
Epoch 3/20
39/39 [==============================] - 23s 583ms/step - loss: 0.2705 - sparse_categorical_accuracy: 0.8395 - val_loss: 0.2423 - val_sparse_categorical_accuracy: 0.8506
Epoch 4/20
39/39 [==============================] - 23s 583ms/step - loss: 0.2444 - sparse_categorical_accuracy: 0.8480 - val_loss: 0.2210 - val_sparse_categorical_accuracy: 0.8536
Epoch 5/20
39/39 [==============================] - 23s 594ms/step - loss: 0.2214 - sparse_categorical_accuracy: 0.8515 - val_loss: 0.2081 - val_sparse_categorical_accuracy: 0.8594
Epoch 6/20
39/39 [==============================] - 23s 591ms/step - loss: 0.2109 - sparse_cate

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

preds = np.array([])
y_trues= np.array([])

# iterate through test set, make predictions based on trained model
for input_example_batch, target_example_batch in ds_series_batch_test:

  pred=model.predict_on_batch(input_example_batch)
  pred_max=tf.argmax(tf.nn.softmax(pred),2).numpy().flatten()
  y_true=target_example_batch.numpy().flatten()

  preds=np.concatenate([preds,pred_max])
  y_trues=np.concatenate([y_trues,y_true])

# remove padding from evaluation
remove_padding = [(p,y) for p,y in zip(preds,y_trues) if y!=0]

r_p = [x[0] for x in remove_padding]
r_t = [x[1] for x in remove_padding]

# print confusion matrix and classification report
print(confusion_matrix(r_p,r_t))
print(classification_report(r_p,r_t))

[[    2     2     0     0     0     0     0     0     0     0     1]
 [   19    38     9     0     0     0     0     0     0     0    10]
 [    2    14  1965    40     0     0     0     0     0   286   377]
 [    0    36    56    91     0     0     0    16     0    27    57]
 [    0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     1     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0    18     0    31    10     6]
 [    0     0   159     0     0     0     8     0    14   629   169]
 [  222    46   764   125    73   224   207    82    80   763 27657]]
              precision    recall  f1-score   support

         1.0       0.01      0.40      0.02         5
         2.0       0.28      0.50      0.36        76
         3.0       0.67      0.73      0.70      2684
         4.0       0.36

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
